<a href="https://colab.research.google.com/github/zzhining/python_ml_dl/blob/main/8_2_RNN_imdb_sentimental_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# imdb sentimental anlaysis

[imdb 영화평 데이터 셋](https://keras.io/api/datasets/imdb/)을 사용하여 영화평 감성분석을 하는 모델을 만들어 봅니다. 레이블은 긍정/부정으로 나뉘어져있으므로 '이진분류'문제입니다.
'분류'문제를 해결하기 위한 Classifier를 모델로 설정하고, 모델 평가는 Accuracy를 사용할 수 있습니다.



## 데이터 불러오기

In [ ]:
from tensorflow.keras.datasets.imdb import load_data

num_words = 20000    # 2만 개의 단어만 학습에 활용
(x_train, y_train), (x_test, y_test) = load_data(num_words = num_words)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

## 데이터 탐색하기

In [ ]:
x_train[0]

In [ ]:
len(x_train[0])

In [ ]:
y_train[0]

In [ ]:
import numpy as np

np.unique(y_train)

In [ ]:
import pandas as pd

df = pd.DataFrame(x_train, columns=['review'])
df['length'] = df['review'].apply(lambda x: len(df['review'][x]))
df['label'] = y_train
df.head()

In [ ]:
import seaborn as sns

sns.histplot(df['length'], kde = True)

In [ ]:
df['length'].describe()

## 데이터 전처리

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

 # 리뷰 문장 중 300 단어만 활용
maxlen = 300
x_train = pad_sequences(x_train, maxlen = maxlen)
x_test = pad_sequences(x_test, maxlen = maxlen)

In [ ]:
x_train.shape, x_test.shape

## 신경망 설계하기

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense

model = Sequential()
# 입력 데이터를 128 차원 벡터로 임베딩
model.add(Embedding(num_words, 128))
# 양방향 LSTM
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Bidirectional(LSTM(128)))
# 분류기
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# 학습 결과를 저장하기 위한 환경 설정정
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)
modelpath="./model/{epoch:02d}-{val_loss:.2f}.keras"

# 학습 중 모니터링 할 조건 설정
checkpointer = ModelCheckpoint(filepath=modelpath,
                               monitor='val_loss',
                               verbose=1,
                               save_best_only=True)

# 학습의 이른 종료를 위한 조건 설정
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
history = model.fit(x_train, y_train, batch_size=256, epochs=50,
                    validation_split=0.2,
                    callbacks=[early_stopping_callback, checkpointer])

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[0:, ['accuracy', 'val_accuracy']].plot()

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
# 모델 구조 plot
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='graph.png')